In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nhl-game-data-20132021/teamInfo.csv
/kaggle/input/nhl-game-data-20132021/playsStats.csv
/kaggle/input/nhl-game-data-20132021/gameStats.csv
/kaggle/input/nhl-game-data-20132021/basicGameStatsSeasonCumulative.csv
/kaggle/input/nhl-game-data-20132021/gamePlayerStats.csv
/kaggle/input/nhl-game-data-20132021/penalties.csv
/kaggle/input/nhl-game-data-20132021/gameGoalieStats.csv
/kaggle/input/nhl-game-data-20132021/gameInfo.csv
/kaggle/input/nhl-game-data-20132021/goals.csv
/kaggle/input/nhl-game-data-20132021/playPlayersStats.csv
/kaggle/input/nhl-game-data-20132021/playerInfo.csv
/kaggle/input/nhl-game-data-20132021/shifts.csv
/kaggle/input/nhl-game-data-20132021/gameOfficialsInfo.csv


<h1> Features </h1>

In [4]:
basicGameStatsSeasonCumulativeDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/basicGameStatsSeasonCumulative.csv')

# One-hot encodings
teams = basicGameStatsSeasonCumulativeDf['Team']
oneHotTeams = pd.get_dummies(teams)
#print(oneHot)

location = basicGameStatsSeasonCumulativeDf['Location']
oneHotLocation = pd.get_dummies(location)
#print(oneHotLocation)

basicGameStatsSeasonCumulativeDf = basicGameStatsSeasonCumulativeDf.drop(['Team', 'Location'], axis=1)
#basicGameStatsSeasonCumulativeDf.insert(2, 'Away', oneHotLocation['Away'])
#basicGameStatsSeasonCumulativeDf.insert(3, 'Home', oneHotLocation['Home'])
basicGameStatsSeasonCumulativeDf = basicGameStatsSeasonCumulativeDf.join(oneHotTeams)
#print(basicGameStatsSeasonCumulativeDf)
basicGameStatsSeasonCumulativeDf = basicGameStatsSeasonCumulativeDf.join(oneHotLocation)
#print(basicGameStatsSeasonCumulativeDf)


# Change label to ints (1 = Win, 0 = Loss)
basicGameStatsSeasonCumulativeDf.loc[basicGameStatsSeasonCumulativeDf['Label'] == 'Win', 'Label'] = 1
basicGameStatsSeasonCumulativeDf.loc[basicGameStatsSeasonCumulativeDf['Label'] == 'Loss', 'Label'] = 0
print(basicGameStatsSeasonCumulativeDf)

print(basicGameStatsSeasonCumulativeDf.shape[0])

           GameID  Fenwick Close %   GF   GA   GD        PP%        PK%  \
0      2013020001        50.000000    0    0    0  50.000000  50.000000   
1      2013020001        50.000000    0    0    0  50.000000  50.000000   
2      2013020002        50.000000    0    0    0  50.000000  50.000000   
3      2013020002        50.000000    0    0    0  50.000000  50.000000   
4      2013020003        50.000000    0    0    0  50.000000  50.000000   
...           ...              ...  ...  ...  ...        ...        ...   
21821  2021021310        55.041518  237  233    4  16.393443  76.888889   
21822  2021021311        42.304718  202  309 -107  14.070352  74.803150   
21823  2021021311        50.011762  262  247   15  24.609375  79.003559   
21824  2021021312        49.649987  213  285  -72  14.611872  74.660633   
21825  2021021312        48.244085  214  261  -47  19.158879  85.135135   

             Sh%        Sv%         PDO  ...  Seattle Kraken  St. Louis Blues  \
0      50.000000  

<h1> Join 2 vectors of each game into 1 </h1>

Home team comes first followed by away team. Label determines result for the home team

In [5]:
def joinGameFeatureVectors(games):
    newVectors = []
    for i in range(0, games.shape[0], 2):
        #print(i)
        #print(games[i])
        newVectors.append(list(games[i]) + list(games[i+1]))
        #print(newVectors)
        #i += 2
    #print(np.array(newVectors))
    return np.array(newVectors)
        
#g = basicGameStatsSeasonCumulativeDf.drop(['GameID','Label'], axis=1).to_numpy()
#res = joinGameFeatureVectors(g)

In [6]:
def joinGameLabels(games):
    newVec = []
    for i in range(0, games.shape[0], 2):
        newVec.append(games[i])
        
    return np.array(newVec)

In [7]:
# 2013 - 2018 = training
#print(basicGameStatsSeasonCumulativeDf)
train = basicGameStatsSeasonCumulativeDf[:14924]
#print(basicGameStatsSeasonCumulativeDf)
xTrain = train.drop(['GameID','Label'], axis=1).to_numpy()
xTrain = joinGameFeatureVectors(xTrain)
#print(xTrain)
#print('----------------------------')
yTrain = train.iloc[:, 12].to_numpy()
#print(yTrain.shape)
yTrain = joinGameLabels(yTrain)
yTrain = yTrain.astype('int')
#print(xTrain)
#print(yTrain.shape)


# 2021 = test
test = basicGameStatsSeasonCumulativeDf[19202:]
xTest = test.drop(['GameID','Label'], axis=1).to_numpy()
#print(xTest.shape)
xTest = joinGameFeatureVectors(xTest)
yTest = test.iloc[:, 12].to_numpy()
#print(yTest.shape)
yTest = joinGameLabels(yTest)
yTest = yTest.astype('int')


#print(xTest2.shape)
#print(yTest2.shape)

<h1> Normalised Data

In [8]:
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()
xTrainNorm = scalar.fit(xTrain).transform(xTrain)
xTestNorm = scalar.transform(xTest)

<h1> Decision Tree

In [11]:
from sklearn import tree
from sklearn.model_selection import cross_val_score


classifier = tree.DecisionTreeClassifier()

print("5-fold avg = " + str(cross_val_score(classifier, xTrain, yTrain, cv=5).mean()))

# Train the classifier
classifier.fit(xTrain, yTrain)

# Predict class of test examples
classifier.predict(xTest)
print('Accuracy = ' + str(classifier.score(xTest, yTest)))
print('Depth = ' + str(classifier.get_depth()))

5-fold avg = 0.5146103622086269
Accuracy = 0.5160060975609756
Depth = 38


<h1> Naive Bayes

In [13]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

gnb = GaussianNB()

print("5-fold avg = " + str(cross_val_score(gnb, xTrain, yTrain, cv=5).mean()))

# Train classifier
gnb.fit(xTrain, yTrain)

# Predict class of test examples
print('Accuracy = ' + str(gnb.score(xTest, yTest)))

5-fold avg = 0.557355774669638
Accuracy = 0.5579268292682927


<h1> SVM

In [21]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score


# Normalise data
pipe = make_pipeline(StandardScaler(), SVC())

print("5-fold avg = " + str(cross_val_score(pipe, xTrain, yTrain, cv=5).mean()))

# Train classifier
pipe.fit(xTrain, yTrain)


# Predict class of test examples
print('Accuracy = ' + str(pipe.score(xTest, yTest)))

5-fold avg = 0.553471338094306
Accuracy = 0.5586890243902439


<h1> Neural Network

In [15]:
from sklearn.neural_network import MLPClassifier

# Normalise data
pipe = make_pipeline(StandardScaler(), MLPClassifier())

print("5-fold avg = " + str(cross_val_score(pipe, xTrain, yTrain, cv=5).mean()))

# Train classifier
pipe.fit(xTrain, yTrain)

# Predict class of test examples
print('Accuracy = ' + str(pipe.score(xTest, yTest)))

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multi

5-fold avg = 0.5201025698119375
Accuracy = 0.5053353658536586


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


<h1> SGD

In [17]:
from sklearn.linear_model import SGDClassifier

# Normalise data
pipe = make_pipeline(StandardScaler(), SGDClassifier())

print("5-fold avg = " + str(cross_val_score(pipe, xTrain, yTrain, cv=5).mean()))

# Train classifier
pipe.fit(xTrain, yTrain)

# Predict class of test examples
print('Accuracy = ' + str(pipe.score(xTest, yTest)))

5-fold avg = 0.5379229074375683
Accuracy = 0.5373475609756098


<h1> AdaBoost

In [18]:
from sklearn.ensemble import AdaBoostClassifier

classifier = AdaBoostClassifier()

print("5-fold avg = " + str(cross_val_score(classifier, xTrain, yTrain, cv=5).mean()))

# Train the classifier
classifier.fit(xTrain, yTrain)

# Predict class of test examples
classifier.predict(xTest)
print('Accuracy = ' + str(classifier.score(xTest, yTest)))

5-fold avg = 0.5478427478366426
Accuracy = 0.5853658536585366


<h1> Gradient Boosting

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier()

print("5-fold avg = " + str(cross_val_score(classifier, xTrain, yTrain, cv=5).mean()))

# Train the classifier
classifier.fit(xTrain, yTrain)

# Predict class of test examples
classifier.predict(xTest)
print('Accuracy = ' + str(classifier.score(xTest, yTest)))

5-fold avg = 0.5477107646227525
Accuracy = 0.6112804878048781


<h1> XGBoost

In [20]:
import xgboost as xgb

classifier = xgb.XGBClassifier()

print("5-fold avg = " + str(cross_val_score(classifier, xTrain, yTrain, cv=5).mean()))

# Train the classifier
classifier.fit(xTrain, yTrain)

# Predict class of test examples
classifier.predict(xTest)
print('Accuracy = ' + str(classifier.score(xTest, yTest)))

5-fold avg = 0.5288164248171539
Accuracy = 0.5594512195121951
